## MLflow's Model Registry

In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [18]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.list_experiments()

[<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [19]:
client.create_experiment(name="my-cool-experiment")

'2'

Let's check the latest versions for the experiment with id `1`...

In [20]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [35]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: f0c5058a460641edb5351ec57b7e84cf, rmse: 6.2862
run id: d8289f4b9b134f9cb09861003376e382, rmse: 6.2908
run id: 8fd59e2d8dce47a29d9148d5b00a546b, rmse: 6.2927
run id: ad9704904a6a48cfb4318b57f8c92ff9, rmse: 6.2937
run id: 06e777dc7fb64cbda525319ecca5fa83, rmse: 6.2950


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [5]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

Register the models from the last 4 runs RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR as an example. 
These models have already saved the model as an artifact. This is required in order to get the model after it is registered.
It turns out that the XGboost (best) model was saved under the folder models_mlflow, and the model itself is not registered (metadata is). It can be registered manually from the ui

In [56]:
runs = client.search_runs(
    experiment_ids='1',
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=4,
    order_by=["attribute.start_time DESC"]
)
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 1a8520773999482881c76b0b02eac595, rmse: 808.7689
run id: 8bedaefee2904778a845108e467c16ee, rmse: 6.9221
run id: 01081828a2944cfb8801f9fc0e4a7352, rmse: 6.7423
run id: e90b70e34d0243859c6ed24c1ffd6b5c, rmse: 6.9089


In [41]:
for run in runs:
    model_uri = f"runs:/{run.info.run_id}/model"
    mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2022/05/25 16:24:40 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 7
Created version '7' of model 'nyc-taxi-regressor'.
Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2022/05/25 16:24:40 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 8
Created version '8' of model 'nyc-taxi-regressor'.
Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2022/05/25 16:24:41 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 9
Created version '9' of model 'nyc-taxi-regr

In [67]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 10, stage: Archived
version: 9, stage: None
version: 12, stage: Production


In [43]:
model_version = 12
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1653488680458, current_stage='Staging', description=None, last_updated_timestamp=1653488754648, name='nyc-taxi-regressor', run_id='1a8520773999482881c76b0b02eac595', run_link=None, source='./mlruns/1/1a8520773999482881c76b0b02eac595/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=7>

In [44]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1653488680458, current_stage='Staging', description='The model version 7 was transitioned to Staging on 2022-05-25', last_updated_timestamp=1653488761569, name='nyc-taxi-regressor', run_id='1a8520773999482881c76b0b02eac595', run_link=None, source='./mlruns/1/1a8520773999482881c76b0b02eac595/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=7>

In [50]:
model_version = 11
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1653488681470, current_stage='Production', description=None, last_updated_timestamp=1653489035817, name='nyc-taxi-regressor', run_id='7c3e9d2729a04e51923fd2b80c675e15', run_link=None, source='./mlruns/1/7c3e9d2729a04e51923fd2b80c675e15/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=11>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [47]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

Select one of the runs in which we saved the dv model as an artifact. This preprocessor is the same for all the runs. Then, download the dv model to the root folder. So unrelated from any run. Destination path is . but the artifact keeps the folder, so it will be stored as ./preprocessor/preprocessor.b
Then, load the dv from the model.

In [24]:
preprocessor_run_id = '01081828a2944cfb8801f9fc0e4a7352'

In [25]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [26]:
client.download_artifacts(run_id=preprocessor_run_id, path='preprocessor', dst_path='.')

'D:\\L\\DataTalks\\mlops-zoomcamp\\02-experiment-tracking\\preprocessor'

In [27]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [28]:
X_test = preprocess(df, dv)

In [29]:
target = "duration"
y_test = df[target].values

In [64]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

CPU times: total: 7.47 s
Wall time: 7.47 s


{'rmse': 6.88524907717107}

In [65]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

[16:39:27] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: total: 21.4 s
Wall time: 3.04 s


{'rmse': 6.250963441118867}

Staging performs better, so change it to production

In [66]:
client.transition_model_version_stage(
    name=model_name,
    version=12,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: creation_timestamp=1653489308363, current_stage='Production', description='', last_updated_timestamp=1653489582551, name='nyc-taxi-regressor', run_id='7c3e9d2729a04e51923fd2b80c675e15', run_link='', source='./mlruns/1/7c3e9d2729a04e51923fd2b80c675e15/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=12>